# Recommender systems (Item/content based)

<div class="alert alert-block alert-info" style="margin-top: 20px">
         <li> Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous, and can be commonly seen in pretty much in everything such as online stores, movies databases and job finders.</li> 
   
</div>
<br>

<li> In order to build a content based recommender system we should build a user profile first, A user profile is something that has user preference of items/content and their associated weights</li>

<li> <a href = "#ref1"> # Lets start by Import libraries </a></li>

In [2]:
#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

<li> <a href = "#ref1"> Loading the data</a></li>

In [4]:
#Storing the movie information into a pandas dataframe
mov_df = pd.read_csv(r"C:\Users\RAGUWING\Documents\moviedataset\ml-latest\movies.csv")
#Storing the user information into a pandas dataframe
rat_df = pd.read_csv(r"C:\Users\RAGUWING\Documents\moviedataset\ml-latest\ratings.csv")
#Head is a function that gets the first N rows of a dataframe. N's default is 5.
mov_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


<li><a href="#ref1">Lets clean up the 'title' feature little bit.</a></li>
<li><a href="#ref1">Remove the year from the 'title' column by using pandas replace function and store in a new year column. cause we only need the movie titles</a></li>

### Perform Data Cleaning and Data Preprosessing 

In [5]:
#Use regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
mov_df['year'] = mov_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
mov_df['year'] = mov_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
mov_df['title'] = mov_df.title.str.replace('(\(\d\d\d\d\))', '')
#strip function to get rid of any ending whitespace characters that may have appeared after extraction.
mov_df['title'] = mov_df['title'].apply(lambda x: x.strip())
mov_df.head()

C:\Users\RAGUWING\AppData\Local\Temp\ipykernel_12224\3023218964.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  mov_df['title'] = mov_df.title.str.replace('(\(\d\d\d\d\))', '')


,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


> <li><a href="#ref1">As you can see the 'genres' feature is categorical values separated by '|', so lets split the values into a list of Genres to simplify for future use. This can be achieved by applying Pythons split string function on the correct column.</a></li>

In [6]:
mov_df['genres'] = mov_df.genres.str.split('|')
mov_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


Since keeping genres in a list format isn't optimal for the content-based recommendation system technique, we will use the One Hot Encoding technique to convert the list of categorical genres to a vector where each column corresponds to one possible genre value of the feature. This encoding is needed for feeding categorical data. In this case, we store every different genre in columns that contain either 1 or 0. 1 shows that a movie has that genre and 0 shows that it doesn't.

In [7]:
#Copying the movie dataframe into a new one
movWithGen_df = mov_df.copy()

#For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column if genre present
for index, row in mov_df.iterrows():
    for genre in row['genres']:
        movWithGen_df.at[index, genre] = 1
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
movWithGen_df = movWithGen_df.fillna(0)
movWithGen_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Next, let's look at the ratings dataframe.


In [8]:
rat_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


Every row in the ratings dataframe has a user id associated with at least one movie, a rating and a timestamp showing when they reviewed it. We won't be needing the timestamp column, so let's drop it to save memory.


In [9]:
#Drop removes a specified row or column from a dataframe
rat_df = rat_df.drop('timestamp', 1)
rat_df.head()

C:\Users\RAGUWING\AppData\Local\Temp\ipykernel_12224\1676709852.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  rat_df = rat_df.drop('timestamp', 1)


,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


# Content-Based recommendation system


Now, let's take a look at how to implement __Content-Based__ or __Item-Item recommendation systems__. This technique attempts to figure out what are the user's favourite preferences and aspects of an item/content is, and then recommends items that has those aspects in it. In our case, we're going to try to figure out the user input's favorite genres from the movies and ratings given, then recommend those movies that has those genres in it.

Let's begin by creating an input user to recommend movies to:


In [11]:
user_Input = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
    {'title':'Akira', 'rating':4.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Waiting to Exhale','rating':3.5}
         ] 
input_Movies = pd.DataFrame(user_Input)
input_Movies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Akira,4.5
3,Jumanji,2.0
4,Pulp Fiction,5.0
5,Waiting to Exhale,3.5


#### Add movieId to user input
With the input complete, let's extract the input movie's ID's from the movies dataframe and add them into it.

We can achieve this by first filtering out the rows that contain the input movie's title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.


In [14]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [16]:
#Filtering out the movies by title
input_Id = mov_df[mov_df['title'].isin(input_Movies['title'].tolist())]
input_Id
#Then merging it so we can get the movieId. It's implicitly merging it by title.
input_Movies = pd.merge(input_Id, input_Movies)
input_Movies
#Dropping information we won't use from the input dataframe
input_Movies = input_Movies.drop('genres', 1).drop('year', 1)
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
input_Movies

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
293,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994
1246,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988
1885,1968,"Breakfast Club, The","[Comedy, Drama]",1985


,movieId,title,genres,year,rating
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,3.5
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,2.0
2,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,3.5
3,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,5.0
4,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988,4.5
5,1968,"Breakfast Club, The","[Comedy, Drama]",1985,5.0


C:\Users\RAGUWING\AppData\Local\Temp\ipykernel_12224\363600521.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  input_Movies = input_Movies.drop('genres', 1).drop('year', 1)
C:\Users\RAGUWING\AppData\Local\Temp\ipykernel_12224\363600521.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  input_Movies = input_Movies.drop('genres', 1).drop('year', 1)


,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,4,Waiting to Exhale,3.5
3,296,Pulp Fiction,5.0
4,1274,Akira,4.5
5,1968,"Breakfast Club, The",5.0


We're going to start by learning the input's preferences, so let's get the subset of movies that the input has watched from the Dataframe containing genres defined with binary values.


In [17]:
#Filtering out the movies from the input
user_Movies = movWithGen_df[movWithGen_df['movieId'].isin(input_Movies['movieId'].tolist())]
user_Movies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
293,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1246,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1885,1968,"Breakfast Club, The","[Comedy, Drama]",1985,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We'll only need the actual genre table, so let's clean this up a bit by resetting the index and dropping the movieId, title, genres and year columns.


In [18]:
#Resetting the index to avoid future issues
user_Movies = user_Movies.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
userGenreTable = user_Movies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

C:\Users\RAGUWING\AppData\Local\Temp\ipykernel_12224\529068926.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  userGenreTable = user_Movies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\RAGUWING\AppData\Local\Temp\ipykernel_12224\529068926.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  userGenreTable = user_Movies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\RAGUWING\AppData\Local\Temp\ipykernel_12224\529068926.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  userGenreTable = user_Movies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\RAGUWING\AppData\Local\Temp\ipykernel_12224\529068926.py:4: FutureWarning: In a futur

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now we're ready to start learning the input's preferences!

To do this, we're going to turn each genre into weights. We can do this by using the input's reviews(ratings) and multiplying them into the input's genre table and then summing up the resulting table by column(each feature). This operation is actually a dot product between a matrix(genre table) and a vector(ratings), so we can simply accomplish by calling the Pandas "dot" function.


In [19]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(input_Movies['rating'])
#The user profile
userProfile

Adventure             10.0
Animation              8.0
Children               5.5
Comedy                17.0
Fantasy                5.5
Romance                3.5
Drama                 13.5
Action                 4.5
Crime                  5.0
Thriller               5.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 4.5
IMAX                   0.0
Documentary            0.0
War                    0.0
Musical                0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

Now, we have the weights for every of the user's preferences. This is known as the User Profile. Using this, we can recommend movies that satisfy the user's preferences.


Let's start by extracting the genre table from the original dataframe:


In [21]:
#Now let's get the genres of every movie in our original dataframe
genreTable = movWithGen_df.set_index(movWithGen_df['movieId'])
#And drop the unnecessary information
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

C:\Users\RAGUWING\AppData\Local\Temp\ipykernel_12224\3447019903.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\RAGUWING\AppData\Local\Temp\ipykernel_12224\3447019903.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\RAGUWING\AppData\Local\Temp\ipykernel_12224\3447019903.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\RAGUWING\AppData\Local\Temp\ipykernel_12224\3447019903.py:4: FutureWarning: In a future version o

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
genreTable.shape

(34208, 20)

* Now that we have our userprofile, and the original genre table from movies df, lets build a recommendation table

In [23]:
#Multiply the genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

movieId
1    0.560976
2    0.256098
3    0.250000
4    0.414634
5    0.207317
dtype: float64

In [24]:
#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Just a peek at the values
recommendationTable_df.head()

movieId
26093     0.768293
27344     0.756098
5018      0.737805
108540    0.701220
1907      0.701220
dtype: float64

* Finally we use the movies id in recommender table and the in the original df. then extract top 20 movies 

In [26]:
# The final recommender table
#The final recommendation table
mov_df.loc[mov_df['movieId'].isin(recommendationTable_df.head(20).keys())]

,movieId,title,genres,year
1824,1907,Mulan,"[Adventure, Animation, Children, Comedy, Drama...",1998
4625,4719,Osmosis Jones,"[Action, Animation, Comedy, Crime, Drama, Roma...",2001
4861,4956,"Stunt Man, The","[Action, Adventure, Comedy, Drama, Romance, Th...",1980
4923,5018,Motorama,"[Adventure, Comedy, Crime, Drama, Fantasy, Mys...",1991
6793,6902,Interstate 60,"[Adventure, Comedy, Drama, Fantasy, Mystery, S...",2002
8605,26093,"Wonderful World of the Brothers Grimm, The","[Adventure, Animation, Children, Comedy, Drama...",1962
9296,27344,Revolutionary Girl Utena: Adolescence of Utena...,"[Action, Adventure, Animation, Comedy, Drama, ...",1999
13250,64645,The Wrecking Crew,"[Action, Adventure, Comedy, Crime, Drama, Thri...",1968
15001,75408,Lupin III: Sweet Lost Night (Rupan Sansei: Swe...,"[Action, Animation, Comedy, Crime, Drama, Myst...",2008
15073,76153,Lupin III: First Contact (Rupan Sansei: Faasut...,"[Action, Animation, Comedy, Crime, Drama, Myst...",2002


### Advantages and Disadvantages of Content-Based Filtering

##### Advantages
* Learns user's input preferences
* Highly personalized for the user
* accurate in recommending for personalized users

##### Disadvantages
* Doesn't take into account what others think of the item, so low quality item recommendations might happen
* Extracting data is not always intuitive
* Determining what characteristics of the item the user dislikes or likes is not always obvious
